In [1]:
import os

In [2]:
%pwd

'c:\\Users\\sridhar\\Desktop\\DATA ANALYTICS PROJECTS\\DATA SCIENCE\\End-To-End-ML-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\sridhar\\Desktop\\DATA ANALYTICS PROJECTS\\DATA SCIENCE\\End-To-End-ML-Project'

In [5]:
import pandas as pd

In [6]:
DATA_PATH='data/winequality-red.csv'
wines = pd.read_csv(DATA_PATH)

In [7]:
wines.head(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [8]:
wines.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [10]:
wines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [11]:
# Purpose: Definition of the DataValidationConfig data class.

from dataclasses import dataclass  # Importing the dataclass decorator.
from pathlib import Path  # Object-oriented interface to filesystem paths.

@dataclass(frozen=True)
class DataValidationConfig:
    """
    Data class for configuration related to data validation.

    Attributes:
        root_dir (Path): Root directory for data validation artifacts.
        STATUS_FILE (str): Path to the status file for recording validation status.
        unzip_data_dir (Path): Directory path of the data file for validation.
        all_schema (dict): Dictionary containing the schema for validation.
    """
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict


In [18]:
# Purpose: Main script or module for the machine learning project.

# Importing constants and utility functions from mlproject package.
from mlproject.constants import *
from mlproject.utils.common import read_yaml,create_directories

In [19]:
# Purpose: Definition of the ConfigurationManager class for managing project configurations.

class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH):
        """
        Initializes the ConfigurationManager with default or provided file paths.

        Args:
            config_filepath (Path, optional): Path to the configuration file. Defaults to CONFIG_FILE_PATH.
            params_filepath (Path, optional): Path to the parameters file. Defaults to PARAMS_FILE_PATH.
            schema_filepath (Path, optional): Path to the schema file. Defaults to SCHEMA_FILE_PATH.
        """
        # Reading configuration, parameters, and schema files using read_yaml function.
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        # Creating the root directory for project artifacts.
        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        """
        Retrieves the configuration for data validation.

        Returns:
            DataValidationConfig: Data class containing data validation configuration.
        """
        # Extracting data validation configuration and schema from the overall project configuration.
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        # Creating the root directory for data validation artifacts.
        create_directories([config.root_dir])

        # Creating a DataValidationConfig object with the extracted configuration.
        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir=config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

In [20]:
import os
from mlproject import logger

In [21]:
# Purpose: Definition of the DataValidation class for handling data validation.

class DataValidation:
    def __init__(self, config: DataValidationConfig):
        """
        Initializes the DataValidation object with the provided configuration.

        Args:
            config (DataValidationConfig): DataValidationConfig object containing data validation settings.
        """
        self.config = config

    def validate_all_columns(self) -> bool:
        """
        Validates all columns in the data file against the specified schema.

        Returns:
            bool: True if all columns are present in the schema, False otherwise.
        """
        try:
            validation_status = None

            # Reading the data file into a pandas DataFrame.
            data = pd.read_csv(self.config.unzip_data_dir)
            
            # Extracting the column names from the data.
            all_cols = list(data.columns)

            # Extracting the expected columns from the schema.
            all_schema = self.config.all_schema.keys()

            # Validating each column against the schema.
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status

        except Exception as e:
            # Handling and re-raising any exceptions that occur during the validation process.
            raise e


In [22]:
# Purpose: Main script or module for the machine learning project.

try:
    # Creating ConfigurationManager to manage project configurations.
    config = ConfigurationManager()

    # Retrieving data validation configuration.
    data_validation_config = config.get_data_validation_config()

    # Creating DataValidation object with the obtained configuration.
    data_validation = DataValidation(config=data_validation_config)

    # Performing data validation.
    data_validation.validate_all_columns()

except Exception as e:
    # Handling and re-raising any exceptions that occur during the process.
    raise e


[2023-12-02 14:53:02,292: INFO: common: YAML file: config\config.yaml loaded successfully]
[2023-12-02 14:53:02,296: INFO: common: YAML file: params.yaml loaded successfully]
[2023-12-02 14:53:02,300: INFO: common: YAML file: schema.yaml loaded successfully]
[2023-12-02 14:53:02,300: INFO: common: Created directory at: artifacts]
[2023-12-02 14:53:02,305: INFO: common: Created directory at: artifacts/data_validation]
